In [1]:
import urllib
import datetime
import requests
import os
import sys
import glob
import re 


from PyPDF2 import PdfFileReader,PdfFileWriter
from bs4 import BeautifulSoup  
from datetime import timedelta, date

def daterange(date1, date2):
    for n in range(int ((date2 - date1).days)+1):
        yield date1 + timedelta(n)
        
start_date = date(2020, 1, 1)
end_date = date(2020, 1, 3)
end_date = end_date  + datetime.timedelta(days=1)
print(f'Start Date is {start_date}')
print(f'End Date is {end_date}')


print ('---------Download Date List  ------')
weekdays = [6,7]
effective_date_list = []
for dt in daterange(start_date, end_date):
    if dt.isoweekday() not in weekdays:
        
        # effective_date is 'datetime.date' type, so transfer to Str
        timestampStr = dt.strftime("%Y-%m-%d")
        effective_date_list.append(timestampStr)
print(effective_date_list)
print ('------------------------------------')

# Generate first page websites list
# http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/html/2020-01/23/node_2.htm
paper_stdaily_base = "http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/html/"
tmp_str='2020-01-23'

paper_fetch_list = []
for idx_date in effective_date_list:
    paper_current_url = paper_stdaily_base + idx_date[0:7] + '/' + idx_date[8:10] + '/node_2.htm'
    paper_fetch_list.append(paper_current_url)

# fetch website of eachday
async def download_one(url):
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as resp: 
            return await resp.text()

async def download_all(effective_date_list, sites):
    tasks = [asyncio.create_task(download_one(site)) for site in sites]
    texts = await asyncio.gather(*tasks)
    
    # generate downloads lists without 404
    download_pdf_name_lists = []
    for date_item, text_item in zip(effective_date_list, texts):
        soup_item = BeautifulSoup(text_item,'html.parser')
        download_tag = soup_item.find_all('title')
        # http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/images/2020-01/23/01/KJRB2020012301.pdf
        # http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/images/2020-01/23/02/KJRB2020012302.pdf
        if '404' not in str(download_tag):
            print(f'{date_item} has news.')
            url_base = 'http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/images/'
            # Download each page of the date
            for page_number in range(1,9):
                download_url = url_base + date_item[0:7] + '/' + date_item[8:10] + '/' + '{:02d}'.format(page_number) + '/'+ 'KJRB' + date_item[0:4] + date_item[5:7] + date_item[8:10] + '{:02d}'.format(page_number) + '.pdf'
                
                current_file_name = 'KJRB' + date_item[0:4] + date_item[5:7] + date_item[8:10] + '{:02d}'.format(page_number) + "_pdf.pdf"
                myfile = requests.get(download_url, allow_redirects=True)
                open(current_file_name, 'wb').write(myfile.content)
                download_pdf_name_lists.append(current_file_name)
                print(f'--- Finish downloading {current_file_name} file')
            pdf_output = PdfFileWriter()
            for infn in download_pdf_name_lists:
                #print infn
                pdf_input = PdfFileReader(open(infn, 'rb'), strict=False)
                # 获取 pdf 共用多少页
                page_count = pdf_input.getNumPages()
                #print(f'The number of pages to merge is : {page_count} page')
                for i in range(page_count):
                    pdf_output.addPage(pdf_input.getPage(i))
            summaryName = date_item[0:4] + date_item[5:7] + date_item[8:10] +'_KJRB_Summary.pdf'
            pdf_output.write(open(summaryName, 'wb'))
            print(f'|========> Finish merge {summaryName} file')
    return

tmp_str1 = 'http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/html/2020-01/23/node_2.htm'
tmp_str2 = 'http://digitalpaper.stdaily.com/http_www.kjrb.com/kjrb/html/2020-01/22/node_2.htm'
#paper_fetch_list = []
#paper_fetch_list.append(tmp_str1)
#paper_fetch_list.append(tmp_str2)

#print(paper_fetch_list)
print(len(paper_fetch_list))
print(len(effective_date_list))
asyncio.run(download_all(effective_date_list, paper_fetch_list)) 

ImportError: cannot import name 'PdfFileReader' from 'PyPDF2' (unknown location)